<a href="https://colab.research.google.com/github/bhnunes/CNNStudy/blob/main/CNNStudy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [35]:
import numpy as np
import json
from keras.applications.vgg16 import VGG16
from keras.preprocessing.image import img_to_array, load_img
from keras.applications.imagenet_utils import preprocess_input

In [36]:
#download VGG16
vgg16=VGG16(weights='imagenet')

In [37]:
vgg16.save('vgg16.h5')

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [38]:
def prepare_image(img_path):
  img=load_img(img_path, target_size=(224,224))
  x=img_to_array(img)
  x=np.expand_dims(x,axis=0)
  x=preprocess_input(x)
  return x

In [39]:
imagePath = "/content/test_image.jpg"
y = prepare_image(imagePath)

In [40]:
preds=vgg16.predict(y)

1/1 [==============================] - 2s 2s/step


In [41]:
CLASS_INDEX=None
CLASS_INDEX_PATH="/content/imagenet_class_index.json"

In [44]:
def get_predictions(preds,top=5):
  global CLASS_INDEX

  CLASS_INDEX=json.load(open(CLASS_INDEX_PATH))

  results=[]
  for pred in preds:
    top_indices=pred.argsort()[-top:][::-1]
    result=[tuple(CLASS_INDEX[str(i)]) + (pred[i],) for i in top_indices]
    result.sort(key=lambda x: x[2], reverse=True)
    results.append(result)
  return results

In [45]:
print(get_predictions(preds,top=5))

[[('n02109047', 'Great_Dane', 0.71246487), ('n02099712', 'Labrador_retriever', 0.13573234), ('n02099267', 'flat-coated_retriever', 0.105223656), ('n02097130', 'giant_schnauzer', 0.013601281), ('n02087394', 'Rhodesian_ridgeback', 0.008055633)]]
